## Sequence modeling of time series
* Recurrent neural nets are popular when the data has a temporal component
* A popular sequence model is known as the "Long Short Term Memory" (LSTM) network
    * LSTM models are able to retain information across long time scales in the input sequence
* Here we will apply the LSTM to our EEG data

## Load the data
* The second and third dimensions are swapped to accommodate the upcoming model

In [14]:
filename = '../data/P300.mat' # the data file
import scipy.io as sio # the library that we use to load Matlab data into Python
data = sio.loadmat(filename)
x_train = np.swapaxes(data['X_train'],1,2)
x_test = np.swapaxes(data['X_test'],1,2)
y_train = data['Y_train']
y_test = data['Y_test']


### Creating the model
* We will work with the Keras LSTM class 
* A key parameter is the number of hidden units in our LSTM layer - here we start with 100 units
* We add a "dropout" parameter to try to mitigate overfitting

In [38]:
import keras
input_shape=(128,64) # this tells Keras the shape of the data that will be passed to it for training/testing
n_classes=2
n_lstm_units=20
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.LSTM(n_lstm_units,dropout=0.2),
        keras.layers.Dense(n_classes, activation="softmax"),
    ]
)
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 20)                6800      
                                                                 
 dense_10 (Dense)            (None, 2)                 42        
                                                                 
Total params: 6,842
Trainable params: 6,842
Non-trainable params: 0
_________________________________________________________________


### Compile the model to prepare for training
* We will use the same training parameters as previously

In [39]:
batch_size = 128  # how many training examples in each "batch" used to compute gradient of loss function
epochs = 30 # how many passes through the data we want the fitting to take
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy",keras.metrics.AUC()])

### Fitting the model
* Note that the LSTM is a complex architecture and is thus heavily parameterized
* We will pay close attention to possible overfitting

In [40]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs)

Epoch 1/30
9/9 [==============================] - 2s 33ms/step - loss: 0.6728 - accuracy: 0.6087 - auc_5: 0.6467
Epoch 2/30
9/9 [==============================] - 0s 34ms/step - loss: 0.5281 - accuracy: 0.7873 - auc_5: 0.8168
Epoch 3/30
9/9 [==============================] - 0s 35ms/step - loss: 0.4830 - accuracy: 0.8204 - auc_5: 0.8412
Epoch 4/30
9/9 [==============================] - 0s 35ms/step - loss: 0.4612 - accuracy: 0.8299 - auc_5: 0.8523
Epoch 5/30
9/9 [==============================] - 0s 36ms/step - loss: 0.4651 - accuracy: 0.8355 - auc_5: 0.8460
Epoch 6/30
9/9 [==============================] - 0s 36ms/step - loss: 0.4467 - accuracy: 0.8365 - auc_5: 0.8595
Epoch 7/30
9/9 [==============================] - 0s 36ms/step - loss: 0.4502 - accuracy: 0.8384 - auc_5: 0.8612
Epoch 8/30
9/9 [==============================] - 0s 34ms/step - loss: 0.4304 - accuracy: 0.8403 - auc_5: 0.8715
Epoch 9/30
9/9 [==============================] - 0s 33ms/step - loss: 0.4320 - accuracy: 0.8374

### Evaluating the sequence model
* The training performance was quite high, but as always, we need to test on unseen data

In [41]:
metrics=model.evaluate(x_test,y_test)
print("Model accuracy = " + str(metrics[1]))
print("Model AUROC = " + str(metrics[2]))

17/17 [==============================] - 1s 6ms/step - loss: 0.4271 - accuracy: 0.8396 - auc_5: 0.8809
Model accuracy = 0.8396226167678833
Model AUROC = 0.8808525800704956


### Interpreting the performance
* Note that while the performance is very good, it did not exceed that obtained with a convolutional model
* It may be argued that the information in EEG is in the motifs, and not embedded in a true sequence (order-dependence)
* In practice, the number of units and dropout parameter are extensively tuned on a "validation" set